<a href="https://colab.research.google.com/github/rakesh0929/basic_vgg19/blob/main/VGG19_BASIC_END_TO_END.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

In [28]:
# !pip install flask-ngrok

In [2]:
tf.__version__

'2.7.0'

In [3]:
from tensorflow.keras.applications.vgg19 import VGG19

In [4]:
model = VGG19()

574726144/574710816 [==============================] - 6s 0us/step


In [5]:
model.summary()

Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [6]:
model.save("model_vgg.h5")

In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [31]:


%%writefile app.py

import numpy as np
import os
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.python.keras.applications.imagenet_utils import decode_predictions
from tensorflow.python.keras.applications.vgg19 import preprocess_input
# from flask_ngrok import run_with_ngrok

from flask import Flask,redirect,url_for,render_template,request

app = Flask(__name__)
# run_with_ngrok(app)
model_path = "/content/model_vgg.h5"

model = load_model(model_path)
model.make_predict_function()

def model_predict(img_path,model):
  img = image.load_img(img_path,target_size=(224,224))
  x = image.img_to_array(img)
  x = np.expand_dims(x,axis=0)
  x = preprocess_input(x)
  preds = model.predict(x)
  return preds

@app.route("/",methods=["GET"])
def index():
  return render_template("/content/drive/MyDrive/project_vgg19/templates/index.html")
@app.route("/predict",methods=["GET","POST"])
def upload():
  if request.method == "POST":
    f = request.files["file"]
    basepath = os.path.dirname(__file__)
    file_path = os.path.join(basepath, '/content/drive/MyDrive/project_vgg19/upload', secure_filename(f.filename))
    f.save(file_path)

    preds = model_predict(file_path,model)

    pred_class = decode_predictions(preds,top=1)
    result = str(pred_class[0][0][1])
    return result
  return None

if __name__ == "__main__":
  app.run(debug=True)

Overwriting app.py


In [32]:
!python "/content/app.py"

2022-02-15 04:42:46.164292: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
 * Serving Flask app "app" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
 * Restarting with stat
2022-02-15 04:42:51.614793: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
 * Debugger is active!
 * Debugger PIN: 166-468-017


In [36]:
pip freeze > requirements.txt